In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas scikit-learn nltk textstat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.3 MB/s eta 0:00:00


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textstat

df = pd.read_csv('/content/drive/Shareddrives/Text mining Project/PROJECT/TWEET 2.csv')
df.head(5)

,Unnamed: 0,id,text,label,type,model_wrong,db.model_preds,status,round,split,annotator
0,1,10000,It should be legal for a husband to rape his wife,hate,notgiven,True,0.97457,dynabench entry,1,train,gnzuctwed3
1,2,10001,dalits are lowlife scum,hate,notgiven,False,0.08233,dynabench entry,1,test,cagnluiznm
2,3,10002,dalits are lowlives,hate,notgiven,True,0.92319,dynabench entry,1,train,cagnluiznm
3,4,10003,It was a better world when women dare not ques...,hate,notgiven,True,0.99006,dynabench entry,1,test,gnzuctwed3
4,5,10004,I don't work this hard so that those immigrant...,hate,notgiven,True,0.98836,dynabench entry,1,train,cagnluiznm


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts = df['text']

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)

print("TF-IDF Feature Matrix:")
print(tfidf_matrix)


TF-IDF Feature Matrix:
  (0, 19888)	0.39690872981688546
  (0, 8545)	0.3264173741664634
  (0, 14703)	0.43726333444390986
  (0, 18296)	0.13003682289838497
  (0, 8849)	0.4172421023813173
  (0, 7126)	0.19401815908866607
  (0, 10392)	0.4597224527327493
  (0, 1759)	0.17501783980148483
  (0, 16329)	0.2173728703325713
  (0, 9683)	0.1638456544164491
  (1, 15915)	0.40287516988486677
  (1, 10795)	0.6578631209909515
  (1, 1136)	0.15298436718962738
  (1, 4550)	0.617659691841611
  (2, 10796)	0.7679620918509951
  (2, 1136)	0.1539873253142313
  (2, 4550)	0.6217090389582678
  (3, 11272)	0.27210572165635705
  (3, 14554)	0.44680733126213784
  (3, 12418)	0.17874007111472978
  (3, 4581)	0.5280423940466298
  (3, 20019)	0.21853138636817976
  (3, 19803)	0.2518574768813646
  (3, 20090)	0.29260664041012857
  (3, 1949)	0.3461628098133684
  :	:
  (40619, 8437)	0.3005300308488855
  (40619, 19690)	0.2372959624858423
  (40619, 7301)	0.2547006409849443
  (40619, 18065)	0.1282369110061293
  (40619, 760)	0.193395608580

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel

X = tfidf_matrix
y = df['label']

# Train logistic regression with L1 regularization
logreg = LogisticRegression(penalty='l1', solver='liblinear')
logreg.fit(X, y)

# Select most important features
model = SelectFromModel(logreg, prefit=True)
X_selected = model.transform(X)


In [ ]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(200, 140, 70), activation='relu', max_iter=1000)

# Train the MLP model
mlp.fit(X_selected, y)

# Print architecture diagram
print("MLP Architecture:")
print("Input Layer: Size =", X_selected.shape[1], "Activation = Sigmoid")
print("Hidden Layers: Sizes =", (200, 140, 70), "Activation = ReLU")
print("Output Layer: Size =", len(set(y)), "Activation = Softmax")


In [ ]:
print("MLP Model Summary:")
print("Input Layer: Size =", X_selected.shape[1], "Activation = Sigmoid")

for i, size in enumerate((200, 140, 70)):
    print(f"Hidden Layer {i + 1}: Size = {size}, Activation = ReLU")

print("Output Layer: Size =", len(set(y)), "Activation = Softmax")


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Train the MLP model
mlp.fit(X_train, y_train)

# Predictions on the test set
y_pred = mlp.predict(X_test)

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.7508923076923077

Classification Report:
              precision    recall  f1-score   support

        hate       0.76      0.79      0.78      4401
     nothate       0.74      0.70      0.72      3724

    accuracy                           0.75      8125
   macro avg       0.75      0.75      0.75      8125
weighted avg       0.75      0.75      0.75      8125



In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate and print sensitivity and specificity
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("\nSensitivity (True Positive Rate):", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.7504

Classification Report:
              precision    recall  f1-score   support

        hate       0.77      0.76      0.77      4401
     nothate       0.72      0.74      0.73      3724

    accuracy                           0.75      8125
   macro avg       0.75      0.75      0.75      8125
weighted avg       0.75      0.75      0.75      8125


Sensitivity (True Positive Rate): 0.7376476906552094
Specificity: 0.761190638491252


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neural_network import MLPClassifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Experiment with different hyperparameters
mlp = MLPClassifier(hidden_layer_sizes=(300, 200, 100), activation='relu', max_iter=1000, learning_rate='adaptive', random_state=42)

# Train the MLP model
mlp.fit(X_train, y_train)

# Predictions on the test set
y_pred = mlp.predict(X_test)

# Print metrics
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Calculate and print sensitivity and specificity
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print("\nSensitivity (True Positive Rate):", sensitivity)
print("Specificity:", specificity)


Accuracy: 0.7496615384615385

Classification Report:
              precision    recall  f1-score   support

        hate       0.76      0.79      0.77      4401
     nothate       0.74      0.70      0.72      3724

    accuracy                           0.75      8125
   macro avg       0.75      0.75      0.75      8125
weighted avg       0.75      0.75      0.75      8125


Sensitivity (True Positive Rate): 0.6987110633727175
Specificity: 0.792774369461486


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
new_tweets = [
    "I love spreading kindness and positivity! #SpreadLove",
    "This is the worst thing ever! I can't believe people are so hateful. #Angry",
    "Why are muslims always fighting with others? Dont they have anything better to do?",
    "I can't stand women who act like they know it all.",
    "This is unacceptable! How can anyone say such hurtful things? #Angry",
    "Gay teachers should not be allowed near children",
    "Why are immigrants stealing all our jobs and money",
    "It's important to address hate speech and work towards building a more inclusive society. #SocialJustice"
]

# Vectorize the new tweets using the same vectorizer used during training
new_tweets_vectorized = tfidf_vectorizer.transform(new_tweets)

if 'model' in globals():
    new_tweets_vectorized = model.transform(new_tweets_vectorized)

# Make predictions on the new tweets
predictions = mlp.predict(new_tweets_vectorized)

# Print the predictions
for tweet, prediction in zip(new_tweets, predictions):
    print(f"Tweet: {tweet}\nPrediction: {prediction}\n")


Tweet: I love spreading kindness and positivity! #SpreadLove
Prediction: nothate

Tweet: This is the worst thing ever! I can't believe people are so hateful. #Angry
Prediction: nothate

Tweet: Why are muslims always fighting with others? Dont they have anything better to do?
Prediction: hate

Tweet: I can't stand women who act like they know it all.
Prediction: hate

Tweet: This is unacceptable! How can anyone say such hurtful things? #Angry
Prediction: nothate

Tweet: Gay teachers should not be allowed near children
Prediction: hate

Tweet: Why are immigrants stealing all our jobs and money
Prediction: hate

Tweet: It's important to address hate speech and work towards building a more inclusive society. #SocialJustice
Prediction: nothate



In [ ]:

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')



# TF-IDF feature matrix
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['text'])
numeric_features = df.drop(['text', 'label'], axis=1)
numeric_features = numeric_features.apply(pd.to_numeric, errors='coerce')
print("TF-IDF Feature Matrix:")
print(pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out()))

# Part-of-Speech (POS) TF feature matrix
def pos_tf_matrix(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    pos_counts = dict()
    total_words = len(tokens)

    for _, pos in pos_tags:
        pos_counts[pos] = pos_counts.get(pos, 0) + 1

    pos_tf = {pos: count / total_words for pos, count in pos_counts.items()}
    return pos_tf

df['pos_tf_matrix'] = df['text'].apply(pos_tf_matrix)
print("\nPart-of-Speech (POS) TF Feature Matrix:")
print(pd.DataFrame(df['pos_tf_matrix'].tolist()).fillna(0))

# Additional features: No of syllables, average syllables per word, number of unique words
df['num_syllables'] = df['text'].apply(textstat.syllable_count)
df['avg_syll_per_word'] = df['text'].apply(lambda x: textstat.syllable_count(x) / len(x.split()))
df['num_unique_words'] = df['text'].apply(lambda x: len(set(x.split())))

print("\nAdditional Features:")
print(df[['num_syllables', 'avg_syll_per_word', 'num_unique_words']])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


TF-IDF Feature Matrix:
        00  000   09   0f  0ff1c3  0ids  0ld  0nly   10  100  ...  špeople  \
0      0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
1      0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
2      0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
3      0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
4      0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
...    ...  ...  ...  ...     ...   ...  ...   ...  ...  ...  ...      ...   
40618  0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
40619  0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
40620  0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
40621  0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   
40622  0.0  0.0  0.0  0.0     0.0   0.0  0.0   0.0  0.0  0.0  ...      0.0   

       šred  šreid  šterf  šthen  šwho  